In [1]:
print("hello")

hello


In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("spark-health-check")
    .getOrCreate()
)

print("Spark version:", spark.version)
print("Spark master:", spark.sparkContext.master)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/26 17:03:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 3.5.1
Spark master: local[*]


In [3]:
data = spark.range(1_000_000)

print("Partitions:", data.rdd.getNumPartitions())
print("Count:", data.count())

Partitions: 4
Count: 1000000
